In [1]:
import numpy as np

In [8]:
theory = np.genfromtxt('tests/sample_files/theory.txt')
binned = np.genfromtxt('tests/sample_files/bin_ordered_data.dat')
noisey = np.genfromtxt('tests/sample_files/noise_mean.txt')

In [9]:
print(np.allclose(binned, noisey, rtol = 0.00000000001))

False


In [10]:
binned_re = binned.reshape(30,-1)
theory_re = theory.reshape(30,-1)

overall_order = ['xi_plus', 'xi_minus']
bin_order = ['bin_1_1', 
             'bin_2_1', 'bin_2_2', 
             'bin_3_1', 'bin_3_2', 'bin_3_3', 
             'bin_4_1', 'bin_4_2', 'bin_4_3', 'bin_4_4', 
             'bin_5_1', 'bin_5_2', 'bin_5_3', 'bin_5_4', 'bin_5_5']
ordered_bins = []
for order in overall_order:
    for bin in bin_order:
        ordered_bins.append(order + '_' + bin)

new_order = [None] * len(ordered_bins)

In [12]:
for i, proper in enumerate(theory_re):
    for j, binned in enumerate(binned_re):
        if np.allclose(proper, binned) is True:
            new_order[i] = ordered_bins[j]